# GRU 모델 시험, W&B sweep 사용 예시

조건식 라벨 데이터를 GRU 레이어를 사용한 모델에서 처리합니다.  
먼저 wandb를 설치합니다. 다음을 Shell에서 실행한 후에 노트북을 다시 켜세요.
  
```shell
pip install wandb
```  

혹은 주피터 노트북에서 다음을 실행하고 주피터 노트북을 다시 켜세요.

```python
!pip install wandb
```

그리고 우측 상단에서 노트북을 trusted로 설정했는데 이게 필수적인지 모르겠습니다.  

In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
import random
import os
import wandb
from wandb.keras import WandbCallback

 ## train 함수
- 이 부분은 독립적으로 실행 가능해야 합니다. 따라서 이 경우 라이브러리 불러오기나 데이터를 불러오는 부분부터 모델을 구성하는 부분까지 다 넣었습니다. 
- 만약 'Run all'로 모델 학습이 되는 주피터 노트북/코랩 파일이라면 그것을 'Download as Python'으로 'py' 파일을 다운받은 후에 그 코드를 통째로 train 함수에 넣어도 됩니다. (들여쓰기가 깨지면 정리해야합니다.) 일반적인 실행

### 추가해야 하는 부분
- wandb.init()넣기 : 여기에 들어갈 default_config 역시 정의합니다. 여기에 wandb에서 기록할 parameter를 다 써줍니다. sweep parameter도 기본값을 다 써야 합니다.  
- model 의 callback 에 들어가는 WandbCallback(): wandb.keras.WandbCallback()   
- wandb.log(): 추가적으로 따져볼 값이 있다면 여기에 넣습니다. 실행(run)마다 출력(print)되는 값들은 wandb 홈페이지에서 그 실행의 'logs'에 들어가면 확인 가능하므로, 따로 넣을 필요는 없습니다.


In [2]:
def train():
    import pandas as pd
    import tensorflow as tf
    from tensorflow import keras
    import matplotlib.pyplot as plt
    import numpy as np
    import wandb
    from wandb.keras import WandbCallback

    def seq_acc(y_true, y_pred):
        y_bin=np.zeros_like(y_pred)
        for i, dd in enumerate(y_bin):
            for j in range(len(dd)):
                pred=y_pred[i][j]
                if pred>=0.5:
                    y_bin[i][j]=1
                else:
                    y_bin[i][j]=0

        predict_true = (y_true == y_bin)

        try:
            score = np.average(np.average(predict_true))
        except ValueError:
            score = mean_squared_error(y_true, y_bin)
        return score

    def my_seq_acc(y_true, y_pred):
        score = tf.py_function(func=seq_acc, inp=[y_true, y_pred], Tout=tf.float32,  name='custom_seq_acc') # tf 2.x
        return score


    class MySeqAccCallback(keras.callbacks.Callback):
        def on_epoch_end(self, epochs, logs=None):
            y_pred=self.model.predict(X_test)
            print('sequence accuracy is {}'.format(seq_acc(y_test, y_pred)))


    default_config={
                         'seq_field':72,
                         'stride_inside_seq':9,
                         'stride_between_seqs':2,
                         'learning_rate':0.01,
                         'split_train_ratio':0.8,
                         'epochs':20,
                         'batch_size':64,
                         'unit_GRU0':64}
    ######### Wandb.init() ##########
    wandb.init(config = default_config)

    locations=['거문도', '울산', '거제도', '통영', '추자도']

    df_merged=pd.read_csv("sensory_preprocessed_df_condition_labeled.csv")
    if df_merged.columns[0]=='Unnamed: 0':
        df_merged = df_merged.iloc[:, 1:]

    print('loaded dataset. Generating sequences')
    seq_length=wandb.config.seq_field//wandb.config.stride_inside_seq
    len_ds=len(df_merged)

    seqs_idx=[]

    start_idx=0
    while start_idx<=len_ds-wandb.config.seq_field:
        seqs_idx.append(list(range(start_idx, start_idx + wandb.config.seq_field, wandb.config.stride_inside_seq
    )))
        start_idx+=wandb.config.stride_between_seqs


    seqs_idx[100],len(seqs_idx[100])

    df_merged.reset_index(inplace=True, drop=True)
    print('Any missing values exist:', df_merged.isna().all().all())

    ds_train_cols=df_merged
    ds_train_cols.reset_index(inplace=True, drop=True)
    print('train dataset columns:',ds_train_cols.columns)

    seq_dataset=np.zeros([len(seqs_idx), len(seqs_idx[0]), len(ds_train_cols.columns)])

    for i, seq in enumerate(seqs_idx):
        for j, row_number in enumerate(seq):
            seq_dataset[i, j]=ds_train_cols.loc[row_number].to_numpy()

    def not_bin_in_occurence(x):
        if x==1 or x==0:
            return x
        else:
            print('exceptional value(not 0 or 1) found. replaced by near one.')
            if x>=0.5:
                return 1
            else:
                return 0
            
    ds_train_cols['적조발생(조건)']=ds_train_cols['적조발생(조건)'].apply(not_bin_in_occurence)


    split_index=int(len(seq_dataset)*wandb.config.split_train_ratio)
    print(split_index, len(seq_dataset))

    train_xy=seq_dataset[:split_index]
    np.random.shuffle(train_xy)
    X_train=train_xy[:,:,0:-1]
    y_train=train_xy[:,:,-1]

    test_xy=seq_dataset[split_index:]
    np.random.shuffle(test_xy)
    X_test=test_xy[:,:,0:-1]
    y_test=test_xy[:,:,-1]

    print(X_train.shape, X_test.shape, y_train.shape, y_test.shape,'\n\n')

    model = keras.Sequential([
        keras.Input(shape=(seq_length, 25)),
        keras.layers.GRU(wandb.config.unit_GRU0),
        keras.layers.Dense(seq_length, activation="sigmoid"),
    ]
    )

    optimizer=keras.optimizers.Adam(learning_rate=wandb.config.learning_rate)

    model.compile(optimizer=optimizer, loss="binary_crossentropy")
    
    ######### WandbCallback ##########
    model.fit(X_train, y_train,
            batch_size=wandb.config.batch_size,
            epochs=wandb.config.epochs, 
            validation_data=(X_test, y_test),
            callbacks=[WandbCallback(training_data = (X_train, y_train),
                                     validation_data = (X_test, y_test)), MySeqAccCallback()])
    y_pred=model.predict(X_test)
    
    ######### Wandb.log() ##########
    wandb.log({"ValidationAcc":seq_acc(y_test, y_pred)})

## wandb.login()
이것을 그대로 실행하면 API 키를 입력하라는 창이 뜹니다. wandb홈페이지에서 우측 상단 프로필 이미지 클릭해서 settings -> dange zone에 보면 API키가 있습니다. API키를 코드에 써두어도 되지만, 보안 문제로 권장되지 않습니다.

In [3]:
wandb.login()

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ········


wandb: Appending key for api.wandb.ai to your netrc file: /aiffel/.netrc


True

## sweep configure
- name은 sweep의 이름으로 적당히 지으면 됩니다.   
- metric: sweep은 '최적의 파라미터 조합'을 찾는 것이고, metric은 어떤 것이 최적인지 판단하는 기준입니다. metric으로는 val_loss 같은 keras 에서 정해진 metric도 되고, wandblog() 에서 직접 설정한 값도 됩니다.   
- parameters: 바꾸어볼 parameter의 범위입니다. "values" 로 지정하면 특정 지정된 값들로만 바꿉니다. "min", "max"를 지정하면 범위 내에서 wandb가 적당한 값을 설정합니다.

In [4]:
sweep_config = {
    "name": "sweep",
    "metric": {
        "goal": "maximize",
        "name": "ValidationAcc"
    },
    "method": "random",
    "parameters": {
        "split_train_ratio":{
            "values":[0.8,]
        },
        "epochs":{
            "values":[1,2,3,4,8,12]
        },
        "learning_rate":{"values":[0.0002, 0.001, 0.0032, 0.01]},
        "unit_GRU0":{"values":[32,64,128]},
        "stride_inside_seq":{"values":[3,9]}
    }
    }

## 실행시 주의사항:
__이 부분은 에러가 떠도 다시 실행시켜 보세요. 이전에 제대로 종료하지 않은 경우 에러가 뜨는 것 같은데, 두 번 실행하면 되는 경우가 있습니다.__  
아래 wandb.agent() 를 실행시키면 이 agent가 위의 설정대로 parameter를 바꿔가며 실행을 계속 합니다.  
project는 wandb 홈페이지에서 이 결과가 저장되는 경로입니다. (지금의 경우 'chhyyi/redzone_GRU_sweep_condition_labeled/sweep_id' 에 저장됩니다.  
__sweep parameter 조합이 무한한 경우(parameter를 범위로 지정한 경우 등) wandb는 계속 실행됩니다. 적당히 한 뒤에 중지시키세요.__


In [6]:
sweep_id = wandb.sweep(sweep_config)

# run the sweep
wandb.agent(sweep_id,
            function=train,
            entity = 'chhyyi',
            project = 'redzone_GRU_sweep_condition_labeled_valacc_goal')

Create sweep with ID: f0g74vf5
Sweep URL: https://wandb.ai/chhyyi/redzone_GRU_sweep_condition_labeled_valacc_goal/sweeps/f0g74vf5


wandb: Agent Starting Run: by29llxw with config:
wandb: 	epochs: 1
wandb: 	learning_rate: 0.0002
wandb: 	split_train_ratio: 0.8
wandb: 	stride_inside_seq: 9
wandb: 	unit_GRU0: 128
wandb: Currently logged in as: chhyyi. Use `wandb login --relogin` to force relogin


loaded dataset. Generating sequences
Any missing values exist: False
train dataset columns: Index(['풍속(m/s)', '풍향(deg)', '기온(°C)', '수온(°C)', '강수량(mm)', '풍속(m/s).1',
       '풍향(deg).1', '기온(°C).1', '수온(°C).1', '강수량(mm).1', '풍속(m/s).2',
       '풍향(deg).2', '기온(°C).2', '수온(°C).2', '강수량(mm).2', '풍속(m/s).3',
       '풍향(deg).3', '기온(°C).3', '수온(°C).3', '강수량(mm).3', '풍속(m/s).4',
       '풍향(deg).4', '기온(°C).4', '수온(°C).4', '강수량(mm).4', '적조발생(조건)'],
      dtype='object')
20106 25133
(20106, 8, 25) (5027, 8, 25) (20106, 8) (5027, 8) 




wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.


315/315 [==============================] - 26s 5ms/step - loss: 0.1887 - val_loss: 0.0214


...

INFO:tensorflow:Assets written to: /aiffel/aiffel/aiffelthon/wandb/run-20221114_081950-3y2zjxgf/files/model-best/assets


INFO:tensorflow:Assets written to: /aiffel/aiffel/aiffelthon/wandb/run-20221114_081950-3y2zjxgf/files/model-best/assets
wandb: Adding directory to artifact (/aiffel/aiffel/aiffelthon/wandb/run-20221114_081950-3y2zjxgf/files/model-best)... Done. 0.0s


sequence accuracy is 0.9966182613885021


ValidationAcc,▁
epoch,▁
loss,▁
val_loss,▁
ValidationAcc,0.99662
best_epoch,0
best_val_loss,0.02188
epoch,0
loss,0.021
val_loss,0.02188


wandb: Ctrl + C detected. Stopping sweep.


### wandb.finish()
저는 사실 이게 뭐 하는 것인지 모릅니다.

In [7]:
wandb.finish()